In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [5]:
import pandas as pd
from utils import helper, config, rayer, kaggle_dataset_helper
from ml.models.ensemble import Ensemble

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common


In [8]:
rayer.get_global_cluster(num_cpus=5)
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-20 09:42:32.926839 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 1 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/3.0 CPU
 0.00/9.313 GiB memory
 0.00/2.702 GiB object_store_memory

Demands:
 (no resource demands)
(scheduler +7s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +7s) Adding 1 node(s) of type small-group.
(scheduler +18s) Resized to 5 CPUs.


In [9]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-20 09:44:21.221736 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 2 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/5.0 CPU
 0.00/16.764 GiB memory
 0.00/4.911 GiB object_store_memory

Demands:
 {'CPU': 1}: 5+ from request_resources()


In [43]:
ray.autoscaler.sdk.request_resources(num_cpus=0)

NameError: name 'ray' is not defined

In [6]:
ds_train, ds_test = kaggle_dataset_helper.get_house_prices_dataset()
ds_train = common.label_encode(ds_train)
ds_test = common.label_encode(ds_test)

ds_train = ds_train.fillna(-1)
ds_test = ds_test.fillna(-1)


df_X = ds_train.loc[:, ds_train.columns != 'SalePrice']
df_y = ds_train['SalePrice']

# X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)

In [7]:

ensemble_set = Ensemble(df_X=df_X,
                        df_y=df_y,
                        n_trials=10, ### for all models
                        boosted_round=10, ### for tree models only
                        epochs=15, ### for DNN models
                        num_
                        ensemble_boosted_round=10, ### for ensemble model which is also a free model
                        ensemble_n_trials=3,
                                )


In [7]:
ensemble_set.fetch_models()

(__run_discoveries__ pid=11203) 2022-11-14T10:18:38PST : INFO : brisk_xgboost : __discover_model__ : 108 : Message : brisk_xgb_1: Starting training for trials:10, boosted rounds: 10, max depth: 10
(__run_discoveries__ pid=11203) /tmp/ray/session_2022-11-13_09-49-14_425708_8/runtime_resources/pip/8c3e19d7a00388f88d63710a2035c24ba84d27d8/virtualenv/lib/python3.9/site-packages/optuna/_optimize.py:93: UserWarning: The default storage cannot be shared by multiple processes. Please use an RDB (RDBStorage) when you use joblib for multi-processing. The usage of RDBStorage can be found in https://optuna.readthedocs.io/en/stable/tutorial/rdb.html.
(__run_discoveries__ pid=11203)   warnings.warn(
(__run_discoveries__ pid=11203) 2022-11-14T10:18:39PST : INFO : brisk_xgboost : __discover_model__ : 119 : Message : brisk_xgb_1: Number of trials: 10
(__run_discoveries__ pid=11203) 2022-11-14T10:18:39PST : INFO : brisk_xgboost : __discover_model__ : 121 : Message : Best trial:4
(__run_discoveries__ pid

In [58]:
# ensemble_set.base_models[1].score

In [59]:
# ensemble_set.load_scores()

In [54]:
ensemble_set.base_model_scores

[[-0.7802736464624289,
  -0.8618265915722401,
 [-0.7802802880703295,
  -0.8618305100182324,
 [0.5804542417246297,
  0.39009188209741097,

In [55]:
ensemble_set.fit()

2022-11-14T12:40:27CST : INFO : ensemble : fit : 140 : Message : Ensemble: fiting ensemble on 3 models
(__run_discoveries__ pid=11203) 2022-11-14T10:40:27PST : INFO : brisk_xgboost : __discover_model__ : 108 : Message : brisk_xgb_ensemble: Starting training for trials:3, boosted rounds: 10, max depth: 10
(__run_discoveries__ pid=11203) /tmp/ray/session_2022-11-13_09-49-14_425708_8/runtime_resources/pip/8c3e19d7a00388f88d63710a2035c24ba84d27d8/virtualenv/lib/python3.9/site-packages/optuna/_optimize.py:93: UserWarning: The default storage cannot be shared by multiple processes. Please use an RDB (RDBStorage) when you use joblib for multi-processing. The usage of RDBStorage can be found in https://optuna.readthedocs.io/en/stable/tutorial/rdb.html.
(__run_discoveries__ pid=11203)   warnings.warn(
(__run_discoveries__ pid=11203) 2022-11-14T10:40:27PST : INFO : brisk_xgboost : __discover_model__ : 119 : Message : brisk_xgb_ensemble: Number of trials: 3
(__run_discoveries__ pid=11203) 2022-11

In [56]:
ensemble_set.score

[-0.7802782707401064, -0.8618296200649376]

In [57]:
ensemble_set.predict(df_X)

array([0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 ,
       0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 ,
       0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 ,
       0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.49861518,
       0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 ,
       0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 ,
       0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 ,
       0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 ,
       0.49861518, 0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 ,
       0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 ,
       0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 ,
       0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 ,
       0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 ,
       0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 ,
       0.5047283 , 0.5047283 , 0.5047283 , 0.5047283 , 0.50472